In [129]:
from google.colab import drive
import os
import torch
import torch.nn as nn
import torchvision.transforms as T
import torch.nn.functional as F
from torchvision import datasets
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Subset
from matplotlib import pyplot as plt
import random
import torch.optim as optim
import time
import numpy as np
import itertools as it

In [130]:
print(f"CUDA is available? {torch.cuda.is_available()}")
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(dev)

CUDA is available? True
cuda


In [131]:
drive.mount("/content/drive",True)
root_dir = "/content/drive/My Drive/SB3/"

Mounted at /content/drive


In [132]:
train_transform = T.Compose([
    T.Resize(320),
    T.ToTensor()
])

test_transform = T.Compose([
    T.Resize(320),
    T.ToTensor()
])

In [133]:
train_dataset = ImageFolder(os.path.join(root_dir, "train"), transform=train_transform)
test_dataset = ImageFolder(os.path.join(root_dir, "test"), transform=test_transform)
num_classes = len(train_dataset.classes)
print(num_classes)

8


In [134]:
print(len(train_dataset))
print(len(test_dataset))

1109
471


In [135]:
x = train_dataset[0][0]
print(x.max())
print(x.min())

tensor(0.9961)
tensor(0.)


In [136]:
class_counts = [train_dataset.targets.count(i) for i in range(num_classes)]
num_train = len(train_dataset)
idx = list(range(num_train))
val_frac = 0.2
idx_iter = iter(idx)
class_idx = [list(it.islice(idx_iter, x)) for x in class_counts]
train_idx = [x[:-int(len(x)*val_frac)] for x in class_idx]
train_idx = list(it.chain.from_iterable(train_idx))
val_idx = [x[-int(len(x)*val_frac):] for x in class_idx]
val_idx = list(it.chain.from_iterable(val_idx))
val_dataset = Subset(train_dataset,val_idx)
train_dataset = Subset(train_dataset, train_idx)

In [138]:
print(len(train_dataset))
print(len(val_dataset))

891
218


In [128]:
'''
num_train = len(train_dataset)
train_idx = list(range(num_train))
random.shuffle(train_idx)
val_frac = 0.3
num_val = int(num_train*val_frac)
num_train = num_train - num_val
val_idx = train_idx[num_train:]
train_idx = train_idx[:num_train]
val_dataset = Subset(train_dataset,val_idx)
train_dataset = Subset(train_dataset, train_idx)
'''

'\nnum_train = len(train_dataset)\ntrain_idx = list(range(num_train))\nrandom.shuffle(train_idx)\nval_frac = 0.3\nnum_val = int(num_train*val_frac)\nnum_train = num_train - num_val\nval_idx = train_idx[num_train:]\ntrain_idx = train_idx[:num_train]\nval_dataset = Subset(train_dataset,val_idx)\ntrain_dataset = Subset(train_dataset, train_idx)\n'

In [ ]:
train_counts = [x-int(x*val_frac) for x in class_counts]
weights = len(train_dataset) / torch.Tensor(train_counts)
weights = weights.to(dev)
print(weights)

In [139]:
train_loader = DataLoader(train_dataset, batch_size=4, num_workers=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, num_workers=4, shuffle=False)
test_loader = DataLoader(test_dataset,   batch_size=4, num_workers=4, shuffle=False)
loaders = {
    "train": train_loader,
    "val" : val_loader,
    "test": test_loader
}

In [140]:
def squash(s, dim=-1):
	'''
	"Squashing" non-linearity that shrunks short vectors to almost zero length and long vectors to a length slightly below 1
	Eq. (1): v_j = ||s_j||^2 / (1 + ||s_j||^2) * s_j / ||s_j||
	
	Args:
		s: 	Vector before activation
		dim:	Dimension along which to calculate the norm
	
	Returns:
		Squashed vector
	'''
	squared_norm = torch.sum(s**2, dim=dim, keepdim=True)
	return squared_norm / (1 + squared_norm) * s / (torch.sqrt(squared_norm) + 1e-8)

In [141]:
class PrimaryCapsules(nn.Module):
  def __init__(self, in_channels, out_channels, vector_length, kernel_size, stride, padding):
    """
    Initialize the layer.
    Args:
      in_channels: 	Number of input channels.
      out_channels: 	Number of output channels.
      vector_length:		Dimensionality, i.e. length, of the output capsule vector.

    """
    super().__init__()
    self.vector_length = vector_length
    self.num_caps_channels = int(out_channels / vector_length)
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)

  def forward(self, x):
    x = self.conv(x)
    x = x.view(x.size(0), self.num_caps_channels, x.size(2), x.size(3), self.vector_length)
    x = x.view(x.size(0), -1, self.vector_length)
    return squash(x)


In [142]:
class RoutingCapsules(nn.Module):
  def __init__(self, in_vector_length, num_in_caps, num_out_caps, out_vector_length, num_routing):
    '''
		Initialize the layer.
		Args:
			in_vector_length: 		Dimensionality (i.e. length) of each capsule vector.
			num_in_caps: 		Number of input capsules if digits layer.
			num_out_caps: 		Number of capsules in the capsule layer
			out_vector_length: 		Dimensionality, i.e. length, of the output capsule vector.
			num_routing:	Number of iterations during routing algorithm		
    '''
    super().__init__()
    self.in_vector_length = in_vector_length
    self.num_in_caps = num_in_caps
    self.num_out_caps = num_out_caps
    self.out_vector_length = out_vector_length
    self.num_routing = num_routing

    self.W = nn.Parameter(torch.randn(1, num_out_caps, num_in_caps, out_vector_length, in_vector_length ) )
  
  def forward(self, x):
    batch_size = x.size(0)
    # (batch_size, num_in_caps, in_vector_length) -> (batch_size, 1, num_in_caps, in_vector_length, 1)
    x = x.unsqueeze(1).unsqueeze(4)
    #
    # W @ x =
    # (1, num_output_caps, num_in_caps, out_vector_length, in_vector_length) @ (batch_size, 1, num_in_caps, in_vector_length, 1) =
    # (batch_size, num_out_caps, num_in_caps, out_vector_length, 1)
    u_hat = torch.matmul(self.W, x)
    # (batch_size, num_out_caps, num_in_caps, out_vector_length)
    u_hat = u_hat.squeeze(-1)
    # detach u_hat during routing iterations to prevent gradients from flowing
    temp_u_hat = u_hat.detach()

    '''
    Procedure 1: Routing algorithm
    '''
    b = torch.zeros(batch_size, self.num_out_caps, self.num_in_caps, 1).to(dev)

    for route_iter in range(self.num_routing-1):
      # (batch_size, num_out_caps, num_in_caps, 1) -> Softmax along num_out_caps
      c = F.softmax(b, dim=1)

      # element-wise multiplication
      # (batch_size, num_out_caps, num_in_caps, 1) * (batch_size, num_in_caps, num_out_caps, out_vector_length) ->
      # (batch_size, num_out_caps, num_in_caps, out_vector_length) sum across num_in_caps ->
      # (batch_size, num_out_caps, out_vector_length)
      s = (c * temp_u_hat).sum(dim=2)
      # apply "squashing" non-linearity along dim_caps
      v = squash(s)
      # dot product agreement between the current output vj and the prediction uj|i
      # (batch_size, num_out_caps, num_in_caps, out_vector_length) @ (batch_size, num_out_caps, out_vector_length, 1)
      # -> (batch_size, num_out_caps, num_in_caps, 1)
      uv = torch.matmul(temp_u_hat, v.unsqueeze(-1))
      b += uv

    # last iteration is done on the original u_hat, without the routing weights update
    c = F.softmax(b, dim=1)
    s = (c * u_hat).sum(dim=2)
    # apply "squashing" non-linearity along dim_caps
    v = squash(s)

    return v

In [143]:
class Reconstruction(nn.Module):
    def __init__(self, num_classes, vector_length):
      super().__init__()
      
      self.fc_layer = nn.Sequential(
        nn.Linear(num_classes*vector_length, 64),
        nn.ReLU()
      )

      self.reconstruction_layers = nn.Sequential(
          nn.ConvTranspose2d(1, 128, kernel_size=5, padding=2, stride=11),
          nn.ReLU(),
          nn.ConvTranspose2d(128, 64, kernel_size=5, padding=2, stride=4, output_padding=(1,1)),
          nn.ReLU(),
          nn.Conv2d(64, 3, kernel_size=5, padding=7, stride=1),
          nn.ReLU()
    )
        
    def forward(self, x):
      x = self.fc_layer(x)
      x = x.view(x.size(0), 1, 8, 8)
      x = self.reconstruction_layers(x)
      return x

In [144]:
class CapsNet(nn.Module):
    def __init__(self, num_classes):
      super().__init__()
      self.conv_layer = nn.Sequential(
          nn.Conv2d(3, 64, kernel_size=5, padding=2, stride=8),
          nn.ReLU()
      )
      self.primary_caps = PrimaryCapsules(in_channels=64, out_channels=128, vector_length=8, kernel_size=5, padding=2, stride=1)
      self.digit_caps = nn.Sequential(
        RoutingCapsules(in_vector_length=8, num_in_caps=25600, num_out_caps=20, out_vector_length=16, num_routing=3),
        RoutingCapsules(in_vector_length=16, num_in_caps=20, num_out_caps=num_classes, out_vector_length=16, num_routing=3)
      )
      self.reconstruction_layer = Reconstruction(num_classes=num_classes, vector_length=16)
    
    '''
    def capsule_average_pooling(self, x):
      height = x.size(3)
      width = x.size(4)
      x = x.sum(dim=4).sum(dim=3)
      x = x / (height * width)
      return x
    '''
    
    def score(self, x):
      return torch.sqrt((x ** 2).sum(dim=2))
    
    def forward(self, x):
      x = self.conv_layer(x)
      x = self.primary_caps(x)
      x = self.digit_caps(x)
      scores = self.score(x)
      x = x.view(x.size(0), x.size(1) * x.size(2))
      reconstructions = self.reconstruction_layer(x)
      return scores, reconstructions

In [145]:
model = CapsNet(num_classes=num_classes)
model = model.to(dev)

In [146]:
batch,labels = next(iter(train_loader))
batch = batch.to(dev)
labels = labels.to(dev)
#print(batch.size())
scores, reconstructions = model(batch)
print(scores.size())
print(reconstructions.size())

torch.Size([4, 8])
torch.Size([4, 3, 320, 320])


In [147]:
batch,labels = next(iter(train_loader))
batch = batch.to(dev)
labels = labels.to(dev)
#print(batch.size())
out = model(batch)

In [148]:
def train(epochs, dev, lr=0.001):
    try:
        # Create model
        model = CapsNet(num_classes=num_classes)
        model = model.to(dev)
        # Optimizer
        optimizer = optim.Adam(model.parameters(), lr=lr)
        # Initialize history
        history_loss = {"train": [], "val": [], "test": []}
        history_accuracy = {"train": [], "val": [], "test": []}
        # Process each epoch
        for epoch in range(epochs):
            # Initialize epoch variables
            sum_loss = {"train": 0, "val": 0, "test": 0}
            sum_accuracy = {"train": 0, "val": 0, "test": 0}
            
            # Process each split
            for split in ["train", "val", "test"]:
                if split == "train":
                  model.train()
                else:
                  model.eval()
                # Process each batch
                for (input, labels) in loaders[split]:
                    # Move to CUDA
                    input = input.to(dev)
                    labels = labels.to(dev)
                    # Reset gradients
                    optimizer.zero_grad()
                    # Compute output
                    pred, reconstructions = model(input)
                    score_loss = F.cross_entropy(pred, labels, weight=weights)
                    reconstruction_loss = F.mse_loss(input, reconstructions)
                    loss = score_loss + reconstruction_loss
                    # Update loss
                    sum_loss[split] += loss.item()
                    # Check parameter update
                    if split == "train":
                        # Compute gradients
                        loss.backward()
                        # Optimize
                        optimizer.step()
                    # Compute accuracy
                    _,pred_labels = pred.max(1)
                    batch_accuracy = (pred_labels == labels).sum().item()/input.size(0)
                    # Update accuracy
                    sum_accuracy[split] += batch_accuracy
            # Compute epoch loss/accuracy
            epoch_loss = {split: sum_loss[split]/len(loaders[split]) for split in ["train", "val", "test"]}
            epoch_accuracy = {split: sum_accuracy[split]/len(loaders[split]) for split in ["train", "val", "test"]}
            # Update history
            for split in ["train", "val", "test"]:
                history_loss[split].append(epoch_loss[split])
                history_accuracy[split].append(epoch_accuracy[split])
            # Print info
            print(f"Epoch {epoch+1}:",
                  f"TrL={epoch_loss['train']:.4f},",
                  f"TrA={epoch_accuracy['train']:.4f},",
                  f"VL={epoch_loss['val']:.4f},",
                  f"VA={epoch_accuracy['val']:.4f},",
                  f"TeL={epoch_loss['test']:.4f},",
                  f"TeA={epoch_accuracy['test']:.4f},"
                )
    except KeyboardInterrupt:
        print("Interrupted")
    finally:
        # Plot loss
        plt.title("Loss")
        for split in ["train", "val", "test"]:
            plt.plot(history_loss[split], label=split)
        plt.legend()
        plt.show()
        # Plot accuracy
        plt.title("Accuracy")
        for split in ["train", "val", "test"]:
            plt.plot(history_accuracy[split], label=split)
        plt.legend()
        plt.show()

In [ ]:
train(100, dev, lr=0.001)

Epoch 1: TrL=2.0520, TrA=0.3255, VL=1.9159, VA=0.4545, TeL=1.9006, TeA=0.4534,
Epoch 2: TrL=2.0117, TrA=0.3980, VL=1.8926, VA=0.3364, TeL=1.8784, TeA=0.3750,
Epoch 3: TrL=2.0209, TrA=0.3729, VL=1.9328, VA=0.3091, TeL=1.9130, TeA=0.3093,
Epoch 4: TrL=2.0327, TrA=0.3274, VL=1.9414, VA=0.3045, TeL=1.9150, TeA=0.3072,
Epoch 5: TrL=2.0156, TrA=0.3049, VL=1.9441, VA=0.3045, TeL=1.9254, TeA=0.3072,
Epoch 6: TrL=1.9737, TrA=0.3154, VL=1.9320, VA=0.5045, TeL=1.9114, TeA=0.4386,
Epoch 7: TrL=1.9797, TrA=0.3558, VL=1.9389, VA=0.4545, TeL=1.9213, TeA=0.4534,
Epoch 8: TrL=1.9773, TrA=0.4301, VL=1.8683, VA=0.5591, TeL=1.8475, TeA=0.4894,
Epoch 9: TrL=1.9259, TrA=0.4387, VL=1.8975, VA=0.5636, TeL=1.8668, TeA=0.5000,
Epoch 10: TrL=1.9277, TrA=0.3419, VL=1.8901, VA=0.3273, TeL=1.8686, TeA=0.3114,
Epoch 11: TrL=1.8925, TrA=0.3868, VL=1.7777, VA=0.4364, TeL=1.7269, TeA=0.4322,
Epoch 12: TrL=1.8308, TrA=0.4436, VL=1.7788, VA=0.3091, TeL=1.7495, TeA=0.3072,
Epoch 13: TrL=1.7797, TrA=0.3307, VL=1.7492, VA=0